# Interactive Synthetic Data Generator
## For Educational Anomaly Detection

In [ ]:
# Import required libraries
import os
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import our generator
from enhanced_data_generator import DarkPoolDataGenerator

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## 1. Generate Dataset with Configurable Parameters

In [ ]:
# Configuration parameters - MODIFY THESE AS NEEDED
NUM_TRANSACTIONS = 100000  # Dataset size
FRAUD_RATE = 0.05          # 5% fraud rate
RANDOM_SEED = 42           # For reproducibility
DIFFICULTY = 'intermediate' # 'beginner', 'intermediate', 'advanced'

# Initialize generator
generator = DarkPoolDataGenerator(random_seed=RANDOM_SEED)

# Generate dataset with fraud patterns
print(f"Generating {NUM_TRANSACTIONS:,} transactions with {FRAUD_RATE*100:.1f}% fraud rate...")
enhanced_df, fraud_metadata = generator.generate_enhanced_dataset(
    num_transactions=NUM_TRANSACTIONS, 
    fraud_rate=FRAUD_RATE
)

# Create data directory if needed
os.makedirs('./data', exist_ok=True)

# Save files
enhanced_df.to_csv('./data/enhanced_raw_data.csv', index=False)
pd.DataFrame(fraud_metadata).to_csv('./data/fraud_patterns_metadata.csv', index=False)

print(f"\nDataset Shape: {enhanced_df.shape}")
print(f"Fraud Patterns Injected: {len(fraud_metadata):,}")
print(f"\nFiles saved to ./data/")

## 2. Generate Student Dataset (No Labels)

In [ ]:
# Generate multiple unique datasets for different student teams
NUM_STUDENT_DATASETS = 5  # Number of teams

# Generate unique datasets
print(f"Generating {NUM_STUDENT_DATASETS} unique student datasets...")
generated_files = generator.generate_student_datasets(
    num_datasets=NUM_STUDENT_DATASETS,
    num_transactions=50000,
    fraud_rate=FRAUD_RATE,
    base_seed=RANDOM_SEED
)

print(f"\n✅ Generated {NUM_STUDENT_DATASETS} unique datasets")
print(f"📁 Files saved in: ./data/student_datasets/")
print(f"🔑 Keep answer keys secure!")

## 3. Data Overview and Basic Statistics

In [ ]:
# Display basic information
print("Dataset Overview:")
print("="*50)
print(f"Total Transactions: {len(enhanced_df):,}")
print(f"Date Range: {enhanced_df['Timestamp'].min()} to {enhanced_df['Timestamp'].max()}")
print(f"\nAmount Statistics:")
print(f"  Min: ${enhanced_df['Amount'].min():,.2f}")
print(f"  Max: ${enhanced_df['Amount'].max():,.2f}")
print(f"  Mean: ${enhanced_df['Amount'].mean():,.2f}")
print(f"  Median: ${enhanced_df['Amount'].median():,.2f}")
print(f"  Std Dev: ${enhanced_df['Amount'].std():,.2f}")
print(f"\nUnique Values:")
print(f"  Merchants: {enhanced_df['Merchant'].nunique()}")
print(f"  Accounts: {enhanced_df['AccountID'].nunique()}")
print(f"  Locations: {enhanced_df['Location'].nunique()}")
print(f"  Transaction Types: {enhanced_df['TransactionType'].nunique()}")

# Display sample data
print("\nSample Transactions:")
enhanced_df.head(10)

## 4. Benford's Law Analysis

In [ ]:
def benford_analysis(data, amount_column='Amount'):
    """Analyze Benford's Law compliance"""
    amounts = data[amount_column].dropna()
    first_digits = amounts.astype(str).str[0].astype(int)
    
    # Calculate observed frequencies
    observed_counts = first_digits.value_counts().sort_index()
    observed_proportions = observed_counts / len(first_digits)
    
    # Calculate expected Benford proportions
    benford_proportions = pd.Series(
        [np.log10(1 + 1/d) for d in range(1, 10)], 
        index=range(1, 10)
    )
    
    # Ensure alignment
    for digit in range(1, 10):
        if digit not in observed_proportions.index:
            observed_proportions[digit] = 0
    observed_proportions = observed_proportions.sort_index()
    
    # Chi-squared test
    expected_counts = benford_proportions * len(first_digits)
    chi_squared = np.sum((observed_counts - expected_counts)**2 / expected_counts)
    p_value = 1 - stats.chi2.cdf(chi_squared, df=8)
    
    return {
        'observed': observed_proportions,
        'expected': benford_proportions,
        'chi_squared': chi_squared,
        'p_value': p_value,
        'compliant': p_value > 0.05
    }

# Perform analysis
benford_results = benford_analysis(enhanced_df)

# Visualize Benford's Law
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

digits = range(1, 10)
width = 0.35

# Distribution comparison
ax1.bar([d - width/2 for d in digits], benford_results['expected'], 
        width, label="Benford's Law", alpha=0.7, color='red')
ax1.bar([d + width/2 for d in digits], benford_results['observed'], 
        width, label='Observed', alpha=0.7, color='blue')
ax1.set_xlabel('First Digit')
ax1.set_ylabel('Proportion')
ax1.set_title('Benford\'s Law Compliance')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Deviation plot
deviations = benford_results['observed'] - benford_results['expected']
colors = ['red' if d > 0 else 'blue' for d in deviations]
ax2.bar(digits, deviations, color=colors, alpha=0.7)
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.5)
ax2.set_xlabel('First Digit')
ax2.set_ylabel('Deviation from Expected')
ax2.set_title('Deviations from Benford\'s Law')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print results
print(f"\nBenford's Law Analysis:")
print(f"Chi-squared: {benford_results['chi_squared']:.2f}")
print(f"P-value: {benford_results['p_value']:.4f}")
if benford_results['compliant']:
    print("✅ Dataset follows Benford's Law (natural distribution)")
else:
    print("⚠️ Dataset deviates from Benford's Law")

## 5. Fraud Pattern Distribution Analysis

In [ ]:
# Load fraud metadata
fraud_df = pd.DataFrame(fraud_metadata)

if len(fraud_df) > 0:
    # Mark fraudulent transactions
    enhanced_df['is_fraud'] = enhanced_df.index.isin(fraud_df['index'])
    
    # Visualize fraud distribution
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. Fraud type distribution
    fraud_types = fraud_df['type'].value_counts()
    axes[0, 0].pie(fraud_types.values, labels=fraud_types.index, autopct='%1.1f%%')
    axes[0, 0].set_title('Fraud Pattern Types Distribution')
    
    # 2. Amount distribution: Normal vs Fraud
    axes[0, 1].hist(enhanced_df[~enhanced_df['is_fraud']]['Amount'], bins=50, 
                    alpha=0.5, label='Normal', color='blue', density=True)
    axes[0, 1].hist(enhanced_df[enhanced_df['is_fraud']]['Amount'], bins=50, 
                    alpha=0.5, label='Fraud', color='red', density=True)
    axes[0, 1].set_xlabel('Amount ($)')
    axes[0, 1].set_ylabel('Density')
    axes[0, 1].set_title('Amount Distribution: Normal vs Fraudulent')
    axes[0, 1].legend()
    axes[0, 1].set_xlim(0, 200000)
    
    # 3. Fraud by merchant
    fraud_by_merchant = enhanced_df.groupby('Merchant')['is_fraud'].mean() * 100
    axes[1, 0].bar(fraud_by_merchant.index, fraud_by_merchant.values)
    axes[1, 0].set_xlabel('Merchant')
    axes[1, 0].set_ylabel('Fraud Rate (%)')
    axes[1, 0].set_title('Fraud Rate by Merchant')
    axes[1, 0].axhline(y=FRAUD_RATE*100, color='red', linestyle='--', alpha=0.5, label='Expected Rate')
    axes[1, 0].legend()
    
    # 4. Temporal pattern
    enhanced_df['Timestamp'] = pd.to_datetime(enhanced_df['Timestamp'])
    enhanced_df['Hour'] = enhanced_df['Timestamp'].dt.hour
    fraud_by_hour = enhanced_df.groupby(['Hour', 'is_fraud']).size().unstack(fill_value=0)
    fraud_by_hour.plot(kind='bar', stacked=True, ax=axes[1, 1], color=['blue', 'red'])
    axes[1, 1].set_xlabel('Hour of Day')
    axes[1, 1].set_ylabel('Transaction Count')
    axes[1, 1].set_title('Transaction Volume by Hour')
    axes[1, 1].legend(['Normal', 'Fraud'])
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"\nFraud Pattern Statistics:")
    print(f"Total Fraudulent Transactions: {enhanced_df['is_fraud'].sum():,}")
    print(f"Actual Fraud Rate: {enhanced_df['is_fraud'].mean()*100:.2f}%")
    print(f"\nFraud Type Breakdown:")
    for fraud_type, count in fraud_types.items():
        print(f"  {fraud_type}: {count} ({count/len(fraud_df)*100:.1f}%)")
else:
    print("No fraud patterns found in dataset")

## 6. Merchant Behavior Profiles

In [ ]:
# Analyze merchant behavior patterns
merchant_profiles = enhanced_df.groupby('Merchant').agg({
    'Amount': ['mean', 'median', 'std', 'count'],
    'TransactionID': 'count'
}).round(2)

merchant_profiles.columns = ['Avg_Amount', 'Median_Amount', 'Std_Amount', 'Transaction_Count', 'Total_Transactions']
merchant_profiles = merchant_profiles.sort_values('Avg_Amount', ascending=False)

# Visualize merchant profiles
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Average transaction amount by merchant
axes[0, 0].bar(merchant_profiles.index, merchant_profiles['Avg_Amount'], color='steelblue')
axes[0, 0].set_xlabel('Merchant')
axes[0, 0].set_ylabel('Average Amount ($)')
axes[0, 0].set_title('Average Transaction Amount by Merchant')

# Transaction volume by merchant
axes[0, 1].bar(merchant_profiles.index, merchant_profiles['Transaction_Count'], color='coral')
axes[0, 1].set_xlabel('Merchant')
axes[0, 1].set_ylabel('Number of Transactions')
axes[0, 1].set_title('Transaction Volume by Merchant')

# Amount distribution boxplot
merchant_amounts = [enhanced_df[enhanced_df['Merchant'] == m]['Amount'].values 
                    for m in sorted(enhanced_df['Merchant'].unique())]
axes[1, 0].boxplot(merchant_amounts, labels=sorted(enhanced_df['Merchant'].unique()))
axes[1, 0].set_xlabel('Merchant')
axes[1, 0].set_ylabel('Amount ($)')
axes[1, 0].set_title('Amount Distribution by Merchant')
axes[1, 0].set_ylim(0, 500000)

# Merchant type classification
merchant_types = {
    'A': 'HFT', 'B': 'Block', 'C': 'Arbitrage', 'D': 'Institutional',
    'E': 'HFT', 'F': 'Block', 'G': 'Retail Agg', 'H': 'Institutional',
    'I': 'Arbitrage', 'J': 'Specialist'
}
type_counts = pd.Series([merchant_types[m] for m in enhanced_df['Merchant']]).value_counts()
axes[1, 1].pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%')
axes[1, 1].set_title('Transaction Volume by Merchant Type')

plt.tight_layout()
plt.show()

print("\nMerchant Profiles:")
print(merchant_profiles)

## 7. Temporal Pattern Analysis

In [ ]:
# Add temporal features
enhanced_df['Timestamp'] = pd.to_datetime(enhanced_df['Timestamp'])
enhanced_df['Hour'] = enhanced_df['Timestamp'].dt.hour
enhanced_df['DayOfWeek'] = enhanced_df['Timestamp'].dt.dayofweek
enhanced_df['Month'] = enhanced_df['Timestamp'].dt.month

# Create temporal heatmap
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Hourly transaction heatmap
hourly_pattern = enhanced_df.pivot_table(
    values='Amount', 
    index='Hour', 
    columns='DayOfWeek', 
    aggfunc='count', 
    fill_value=0
)
sns.heatmap(hourly_pattern, cmap='YlOrRd', annot=False, ax=axes[0, 0])
axes[0, 0].set_title('Transaction Volume Heatmap (Hour x Day of Week)')
axes[0, 0].set_xlabel('Day of Week (0=Monday)')
axes[0, 0].set_ylabel('Hour of Day')

# Average amount by hour
hourly_avg = enhanced_df.groupby('Hour')['Amount'].mean()
axes[0, 1].plot(hourly_avg.index, hourly_avg.values, marker='o', linewidth=2)
axes[0, 1].set_xlabel('Hour of Day')
axes[0, 1].set_ylabel('Average Amount ($)')
axes[0, 1].set_title('Average Transaction Amount by Hour')
axes[0, 1].grid(True, alpha=0.3)

# Fraud concentration by hour (if fraud data exists)
if 'is_fraud' in enhanced_df.columns:
    fraud_by_hour = enhanced_df.groupby('Hour')['is_fraud'].mean() * 100
    axes[1, 0].bar(fraud_by_hour.index, fraud_by_hour.values, 
                   color=['red' if h in [2, 3, 4, 22, 23] else 'steelblue' 
                          for h in fraud_by_hour.index])
    axes[1, 0].set_xlabel('Hour of Day')
    axes[1, 0].set_ylabel('Fraud Rate (%)')
    axes[1, 0].set_title('Fraud Rate by Hour (Red = Suspicious Hours)')
    axes[1, 0].axhline(y=FRAUD_RATE*100, color='black', linestyle='--', alpha=0.5)
else:
    axes[1, 0].text(0.5, 0.5, 'No fraud data available', 
                    ha='center', va='center', transform=axes[1, 0].transAxes)

# Transaction type distribution over time
type_by_hour = enhanced_df.pivot_table(
    values='Amount',
    index='Hour',
    columns='TransactionType',
    aggfunc='count',
    fill_value=0
)
type_by_hour.plot(kind='area', stacked=True, ax=axes[1, 1], alpha=0.7)
axes[1, 1].set_xlabel('Hour of Day')
axes[1, 1].set_ylabel('Transaction Count')
axes[1, 1].set_title('Transaction Types by Hour')
axes[1, 1].legend(title='Type')

plt.tight_layout()
plt.show()

# Print temporal statistics
print("\nTemporal Pattern Statistics:")
print(f"Peak Hour: {hourly_avg.idxmax()}:00 (${hourly_avg.max():,.2f} avg)")
print(f"Quiet Hour: {hourly_avg.idxmin()}:00 (${hourly_avg.min():,.2f} avg)")
if 'is_fraud' in enhanced_df.columns:
    print(f"Highest Fraud Hour: {fraud_by_hour.idxmax()}:00 ({fraud_by_hour.max():.2f}%)")

## 8. Statistical Anomaly Detection Preview

In [ ]:
# Calculate anomaly scores using simple statistical methods
from scipy.stats import zscore

# Z-score for amounts
enhanced_df['amount_zscore'] = np.abs(zscore(enhanced_df['Amount']))

# Identify statistical outliers
outliers = enhanced_df[enhanced_df['amount_zscore'] > 3]

# Visualize outliers
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Scatter plot of amounts with outliers highlighted
axes[0].scatter(range(len(enhanced_df)), enhanced_df['Amount'], 
                alpha=0.1, s=1, label='Normal')
axes[0].scatter(outliers.index, outliers['Amount'], 
                color='red', s=10, label='Statistical Outliers')
axes[0].set_xlabel('Transaction Index')
axes[0].set_ylabel('Amount ($)')
axes[0].set_title('Transaction Amounts with Statistical Outliers')
axes[0].legend()
axes[0].set_ylim(0, enhanced_df['Amount'].quantile(0.99))

# Distribution of z-scores
axes[1].hist(enhanced_df['amount_zscore'], bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(x=3, color='red', linestyle='--', label='Outlier Threshold')
axes[1].set_xlabel('Z-Score')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Amount Z-Scores')
axes[1].legend()

plt.tight_layout()
plt.show()

# Compare outliers with actual fraud (if available)
print(f"\nStatistical Anomaly Detection Results:")
print(f"Total Statistical Outliers: {len(outliers):,} ({len(outliers)/len(enhanced_df)*100:.2f}%)")

if 'is_fraud' in enhanced_df.columns:
    outliers['is_fraud'] = outliers.index.isin(fraud_df['index'])
    true_positives = outliers['is_fraud'].sum()
    false_positives = len(outliers) - true_positives
    total_fraud = enhanced_df['is_fraud'].sum()
    false_negatives = total_fraud - true_positives
    
    precision = true_positives / len(outliers) if len(outliers) > 0 else 0
    recall = true_positives / total_fraud if total_fraud > 0 else 0
    
    print(f"\nSimple Statistical Method Performance:")
    print(f"  True Positives: {true_positives}")
    print(f"  False Positives: {false_positives}")
    print(f"  False Negatives: {false_negatives}")
    print(f"  Precision: {precision:.2%}")
    print(f"  Recall: {recall:.2%}")
    print(f"\nNote: This is a simple baseline. Advanced methods will perform better.")

## 9. Feature Correlation Analysis

In [ ]:
# Prepare features for correlation analysis
correlation_df = enhanced_df.copy()
correlation_df['Merchant_num'] = correlation_df['Merchant'].map(dict(zip('ABCDEFGHIJ', range(10))))
correlation_df['Location_num'] = pd.Categorical(correlation_df['Location']).codes
correlation_df['Type_num'] = pd.Categorical(correlation_df['TransactionType']).codes

# Select numerical features
numerical_features = ['Amount', 'AccountID', 'Merchant_num', 'Location_num', 
                      'Type_num', 'Hour', 'DayOfWeek']
if 'is_fraud' in correlation_df.columns:
    correlation_df['is_fraud_num'] = correlation_df['is_fraud'].astype(int)
    numerical_features.append('is_fraud_num')

# Calculate correlation matrix
correlation_matrix = correlation_df[numerical_features].corr()

# Visualize correlation heatmap
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, fmt='.2f', 
            cmap='coolwarm', center=0, vmin=-1, vmax=1)
plt.title('Feature Correlation Heatmap')
plt.tight_layout()
plt.show()

# Print strongest correlations
if 'is_fraud_num' in numerical_features:
    fraud_correlations = correlation_matrix['is_fraud_num'].drop('is_fraud_num').abs().sort_values(ascending=False)
    print("\nTop Features Correlated with Fraud:")
    for feature, corr in fraud_correlations.head(5).items():
        print(f"  {feature}: {corr:.3f}")

## 10. Export Summary Report for Teachers

In [ ]:
# Generate comprehensive summary report
report = {
    'Dataset Overview': {
        'Total Transactions': len(enhanced_df),
        'Date Range': f"{enhanced_df['Timestamp'].min()} to {enhanced_df['Timestamp'].max()}",
        'Unique Merchants': enhanced_df['Merchant'].nunique(),
        'Unique Accounts': enhanced_df['AccountID'].nunique(),
    },
    'Amount Statistics': {
        'Min': f"${enhanced_df['Amount'].min():,.2f}",
        'Max': f"${enhanced_df['Amount'].max():,.2f}",
        'Mean': f"${enhanced_df['Amount'].mean():,.2f}",
        'Median': f"${enhanced_df['Amount'].median():,.2f}",
        'Std Dev': f"${enhanced_df['Amount'].std():,.2f}"
    },
    'Benford Analysis': {
        'Chi-squared': benford_results['chi_squared'],
        'P-value': benford_results['p_value'],
        'Compliant': benford_results['compliant']
    }
}

if 'is_fraud' in enhanced_df.columns:
    report['Fraud Statistics'] = {
        'Total Fraud Cases': enhanced_df['is_fraud'].sum(),
        'Fraud Rate': f"{enhanced_df['is_fraud'].mean()*100:.2f}%",
        'Fraud Types': fraud_df['type'].value_counts().to_dict()
    }

# Save report
import json
with open('./data/teacher_summary_report.json', 'w') as f:
    json.dump(report, f, indent=2, default=str)

print("\n" + "="*50)
print("TEACHER SUMMARY REPORT")
print("="*50)
for section, data in report.items():
    print(f"\n{section}:")
    for key, value in data.items():
        print(f"  {key}: {value}")

print("\n" + "="*50)
print("Report saved to: ./data/teacher_summary_report.json")
print("\nFiles Generated:")
print("  - enhanced_raw_data.csv (full dataset)")
print("  - fraud_patterns_metadata.csv (answer key - DO NOT SHARE)")
print(f"  - student_dataset_{DIFFICULTY}.csv (unlabeled for students)")
print(f"  - teacher_answer_key_{DIFFICULTY}.csv (grading reference)")
print("  - teacher_summary_report.json (this report)")

## 11. Grade Student Submissions (Teacher Only)

In [ ]:
# Example: Grade a student submission using grading tools
from grading_tools import evaluate_detection_performance, analyze_pattern_detection, generate_grade_report

# Simulate a student submission (in practice, load actual student CSV)
# For demo, we'll create a simple prediction
student_predictions = enhanced_df.copy()[['TransactionID']]
student_predictions['is_fraud'] = 0  # Initialize all as normal

# Simple detection: flag high amounts as fraud (naive approach)
threshold = enhanced_df['Amount'].quantile(0.98)
student_predictions.loc[enhanced_df['Amount'] > threshold, 'is_fraud'] = 1

# Save simulated submission
student_predictions.to_csv('./data/sample_submission.csv', index=False)

# Grade the submission
print("Grading submission...")
results = evaluate_detection_performance(
    './data/sample_submission.csv',
    './data/fraud_patterns_metadata.csv'
)

# Generate grade report
report = generate_grade_report(results)
print(report)

# Analyze pattern detection
print("\nPattern-Specific Detection:")
pattern_analysis = analyze_pattern_detection(
    './data/sample_submission.csv',
    './data/fraud_patterns_metadata.csv'
)
print(pattern_analysis)